In [8]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json
import os

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width' , 400)

In [9]:
# Variables
season = "2021-22"

source_path = os.getcwd()

cleaned_data_path =  os.path.join(source_path, "data" , season , "Cleaned_data")
merged_data_path =  os.path.join(source_path, "data" , season , "Merged_data")


In [10]:
all_players = pd.read_csv (os.path.join(cleaned_data_path, 'all_players.csv'), parse_dates= ["news_added"])

all_teams = pd.read_csv (os.path.join(cleaned_data_path, 'all_teams.csv'))
all_events = pd.read_csv (os.path.join(cleaned_data_path, 'all_events.csv'), parse_dates= ["deadline_time"])
player_types = pd.read_csv(os.path.join(cleaned_data_path, 'player_types.csv'))
game_phases = pd.read_csv(os.path.join(cleaned_data_path, 'game_phases.csv'))

player_season_history = pd.read_csv(os.path.join(cleaned_data_path, 'player_season_history.csv'))
player_gameweek_history = pd.read_csv(os.path.join(cleaned_data_path, 'player_gameweek_history.csv') , parse_dates = ['kickoff_time'])
player_future_fixture = pd.read_csv(os.path.join(cleaned_data_path, 'player_future_fixture.csv'), parse_dates = ["kickoff_time"])


In [11]:
all_teams.head()

,team_code,draw,team_form,team_id,loss,team_name,played,points,position,team_short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,NaN,1,0,Arsenal,0,0,0,ARS,...,NaN,False,0,1210,1270,1150,1210,1190,1220,1
1,7,0,NaN,2,0,Aston Villa,0,0,0,AVL,...,NaN,False,0,1130,1160,1140,1110,1090,1090,2
2,94,0,NaN,3,0,Brentford,0,0,0,BRE,...,NaN,False,0,1060,1070,1120,1150,1080,1120,130
3,36,0,NaN,4,0,Brighton,0,0,0,BHA,...,NaN,False,0,1130,1130,1160,1160,1100,1120,131
4,90,0,NaN,5,0,Burnley,0,0,0,BUR,...,NaN,False,0,1060,1050,1080,1130,1060,1100,43


In [12]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,...,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,PP90,PPMM,VAPM
0,100.0,100.0,80201,0,0,-5,5,0,1,0.0,...,30,NaN,NaN,NaN,NaN,NaN,NaN,1.333333,0.288889,-0.155556
1,0.0,0.0,115918,0,0,0,0,0,1,0.0,...,53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.500000
2,0.0,0.0,47431,0,0,-2,2,0,3,0.0,...,292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.317460
3,0.0,0.0,54694,0,0,-4,4,0,4,0.0,...,21,NaN,NaN,NaN,NaN,NaN,NaN,3.822394,0.322917,0.114583
4,100.0,100.0,58822,0,0,-3,3,0,2,0.0,...,126,NaN,NaN,NaN,NaN,NaN,NaN,2.302772,0.404762,-0.071429


In [13]:
player_future_fixture.head()

,fixture_id,fixture_code,home_team_id,team_h_score,away_team_id,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id,opponent_team_id,team_id
0,279,2210549,18,NaN,1,NaN,28.0,False,0,False,2022-03-06 14:00:00,Gameweek 28,False,2,1,18,1
1,281,2210551,1,NaN,9,NaN,29.0,False,0,False,2022-03-13 16:30:00,Gameweek 29,True,3,1,9,1
2,261,2210531,1,NaN,11,NaN,29.0,False,0,False,2022-03-16 20:15:00,Gameweek 29,True,4,1,11,1
3,291,2210561,2,NaN,1,NaN,30.0,False,0,False,2022-03-19 12:30:00,Gameweek 30,False,3,1,2,1
4,304,2210574,7,NaN,1,NaN,31.0,False,0,False,2022-04-04 19:00:00,Gameweek 31,False,2,1,7,1


# Merge to all_players

In [14]:
df_merge = all_teams[["team_code" , "team_strength" , "team_short_name" , "strength_overall_home" , "strength_overall_away"]]
all_players = all_players.merge(df_merge, on = 'team_code')

df_merge = player_types[['player_type_id' , 'position_name_short']]
all_players = all_players.merge(df_merge , on = "player_type_id")


In [15]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,100.0,100.0,80201,0,0,-5,5,0,1,0.0,...,NaN,NaN,1.333333,0.288889,-0.155556,4,ARS,1210,1270,GKP
1,0.0,0.0,115918,0,0,0,0,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.500000,4,ARS,1210,1270,GKP
2,0.0,0.0,463748,0,0,0,0,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.500000,4,ARS,1210,1270,GKP
3,NaN,NaN,225321,0,0,6,-6,3,1,0.0,...,NaN,NaN,5.026596,0.980392,0.588235,4,ARS,1210,1270,GKP
4,NaN,NaN,220682,0,0,-1,1,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.512821,4,ARS,1210,1270,GKP


# Merge to player_gameweek_history

In [16]:
for i in all_players.columns:
    print(i)

chance_of_playing_next_round
chance_of_playing_this_round
player_code
cost_change_event
cost_change_event_fall
cost_change_start
cost_change_start_fall
dreamteam_count
player_type_id
ep_next
ep_this
event_points
first_name
form
player_id
in_dreamteam
news
news_added
now_cost
photo
PPG
second_name
selected_by_percent
status
team_id
team_code
total_points
transfers_in
transfers_in_event
transfers_out
transfers_out_event
value_form
value_season
web_name
total_minutes
goals_scored
assists
clean_sheets
goals_conceded
own_goals
penalties_saved
penalties_missed
yellow_cards
red_cards
saves
bonus
bps
influence
creativity
threat
ict_index
influence_rank
influence_rank_type
creativity_rank
creativity_rank_type
threat_rank
threat_rank_type
ict_index_rank
ict_index_rank_type
corners_and_indirect_freekicks_order
corners_and_indirect_freekicks_text
direct_freekicks_order
direct_freekicks_text
penalties_order
penalties_text
PP90
PPMM
VAPM
team_strength
team_short_name
strength_overall_home
strength_o

In [17]:
player_gameweek_history.head()

,player_id,fixture_id,opponent_team_id,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,1,3,1,False,2021-08-13 19:00:00,2,0,1,90,...,11,11.8,0.0,0.0,1.2,50,0,153122,0,0
1,1,18,6,2,True,2021-08-22 15:30:00,0,2,2,90,...,17,24.2,0.0,0.0,2.4,50,-25816,144885,2627,28443
2,1,24,12,1,False,2021-08-28 11:30:00,5,0,3,90,...,20,43.0,0.0,0.0,4.3,49,-14559,136757,5164,19723
3,1,31,15,0,True,2021-09-11 14:00:00,1,0,4,0,...,0,0.0,0.0,0.0,0.0,49,6564,153130,35360,28796
4,1,43,5,0,False,2021-09-18 14:00:00,0,1,5,0,...,0,0.0,0.0,0.0,0.0,49,-23847,129851,4822,28669


In [18]:
df_merge = all_players[["player_id" , "player_type_id" , "PPG" , "PP90" , "team_strength" , "team_id"]]
player_gameweek_history = player_gameweek_history.merge(df_merge , on = "player_id")

player_gameweek_history.head()

,player_id,fixture_id,opponent_team_id,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,value,transfers_balance,selected,transfers_in,transfers_out,player_type_id,PPG,PP90,team_strength,team_id
0,1,1,3,1,False,2021-08-13 19:00:00,2,0,1,90,...,50,0,153122,0,0,1,1.3,1.333333,4,1
1,1,18,6,2,True,2021-08-22 15:30:00,0,2,2,90,...,50,-25816,144885,2627,28443,1,1.3,1.333333,4,1
2,1,24,12,1,False,2021-08-28 11:30:00,5,0,3,90,...,49,-14559,136757,5164,19723,1,1.3,1.333333,4,1
3,1,31,15,0,True,2021-09-11 14:00:00,1,0,4,0,...,49,6564,153130,35360,28796,1,1.3,1.333333,4,1
4,1,43,5,0,False,2021-09-18 14:00:00,0,1,5,0,...,49,-23847,129851,4822,28669,1,1.3,1.333333,4,1


## Save to file

In [19]:
all_players.to_csv (os.path.join(merged_data_path,'all_players_merged.csv'), index = False, header=True)
player_gameweek_history.to_csv(os.path.join(merged_data_path, 'player_gameweek_history.csv') , index = False, header=True)
